<a href="https://colab.research.google.com/github/dtandev/dw_matrix_2/blob/master/day4_5_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 4.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.7MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/Matrix/dw_matrix_2"

/content/drive/My Drive/Colab Notebooks/Matrix/dw_matrix_2


In [4]:
ls

data/  LICENSE  README.md


In [0]:
df = pd.read_hdf('data/car.h5')

In [0]:
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]
  if '__cat' in feat: continue
  df[feat+'__cat'] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if '__cat' in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

In [0]:
X = df[cat_feats].values
y =df['price_value'].values

In [9]:
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv = 3, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-19695.13091100928, 148.72570644015937)

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv = 3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [11]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

In [12]:
model = RandomForestRegressor(max_depth= 5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [13]:
xgb_params = {
    'max_depth' : 4, 
    'n_estimators' : 40, 
    'learning_rate' : 0.1, 
    'seed' : 0}
model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats)

[16:21:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:21:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:21:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-14759.16656690881, 145.98511642132516)

In [14]:

model.fit(X, y)
imp = PermutationImportance(model, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names = cat_feats)

[16:22:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1268 ± 0.0024,param_napęd__cat
0.1141 ± 0.0019,param_stan__cat
0.0892 ± 0.0027,param_rok-produkcji__cat
0.0682 ± 0.0024,param_skrzynia-biegów__cat
0.0637 ± 0.0006,param_faktura-vat__cat
0.0428 ± 0.0008,param_moc__cat
0.0242 ± 0.0009,param_typ__cat
0.0222 ± 0.0013,feature_kamera-cofania__cat
0.0164 ± 0.0006,param_pojemność-skokowa__cat
0.0157 ± 0.0009,seller_name__cat


In [15]:
feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_marka-pojazdu__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_system-start-stop__cat',
'param_wersja__cat',
'param_model-pojazdu__cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[16:26:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-15404.124271934847, 136.90658828574513)

In [16]:
df['param_napęd'].unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int(x))

feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_marka-pojazdu__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_system-start-stop__cat',
'param_wersja__cat',
'param_model-pojazdu__cat']

In [18]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[16:26:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-12765.33650109561, 103.56851040928021)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x)== 'None' else int(x.split(' ')[0]))


In [0]:

feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'param_marka-pojazdu__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_system-start-stop__cat',
'param_wersja__cat',
'param_model-pojazdu__cat']

In [21]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[16:26:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:26:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10876.74690040911, 54.986859009710194)

In [22]:
df['param_pojemność-skokowa']

0           898 cm3
2         1 560 cm3
3         3 000 cm3
4         1 560 cm3
5         1 984 cm3
            ...    
160609    1 120 cm3
160610    1 200 cm3
160611    1 900 cm3
160614    2 500 cm3
160615    1 360 cm3
Name: param_pojemność-skokowa, Length: 106494, dtype: object

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)== 'None' else int(str(x).split('cm')[0].replace(' ', '')))


In [24]:
df['param_pojemność-skokowa']

0          898
2         1560
3         3000
4         1560
5         1984
          ... 
160609    1120
160610    1200
160611    1900
160614    2500
160615    1360
Name: param_pojemność-skokowa, Length: 106494, dtype: int64

In [0]:
feats = ['param_napęd__cat',
'param_stan__cat',
'param_rok-produkcji',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_typ__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'param_marka-pojazdu__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_wspomaganie-kierownicy__cat',
'param_kod-silnika__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_system-start-stop__cat',
'param_wersja__cat',
'param_model-pojazdu__cat']

In [26]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[16:26:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:27:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:27:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10722.267820510373, 104.33135317639517)

## Day 5

In [27]:
!pip install hyperopt

In [0]:
from hyperopt import hp, fmin, tpe, STATUS_OK

In [33]:
def obj_func(params):
  print('Training with params: ')
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}


xgb_reg_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100, 
    'seed': 0

}

best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.25, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.3, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.3, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.15000000000000002, 'max_depth': 14, 'n_es

{'colsample_bytree': 0.8,
 'learning_rate': 1,
 'max_depth': 7,
 'subsample': 0.9500000000000001}